In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
import pickle
import os
from moviepy.editor import VideoFileClip
% matplotlib inline

In [2]:
# calibrate camera
class Camera():
    def __init__(self, num_xpts, num_ypts, debug_mode=False):
        # number of x points in test images
        self.num_xpts = num_xpts
        # number of y points in test images
        self.num_ypts = num_ypts
        # Camera matrix
        self.mtx = None
        # distortion coeff
        self.dist = None
        # Camera rotation vectors
        self.rvecs = None 
        #Camera translation vectors
        self.tvecs = None
        # set the default debug mode
        self.debug_mode = debug_mode
        # source & destination cordinates for perspective transform (found manually)
        self.source_cord = np.float32([[608, 445], [669, 441], [1091, 714], [226, 711]])
        self.dest_cordinates = np.float32([[200,0], [1080,0], [1080,719], [200,719]])
        
        
        # perspective transform matrix
        self.M = cv2.getPerspectiveTransform(self.source_cord, self.dest_cordinates)
        # inverse transform
        self.Minv = cv2.getPerspectiveTransform(self.dest_cordinates, self.source_cord)
        
    
    def caliberate_camera(self, files):
        print("Caliberating the Camera ...")
        images = glob.glob(files)
        
        objpoints = [] # 3D points in real world space
        imgpoints = [] # 2D points in image plane
        
        # object points .. (0,0,0), (1,0,0),...
        objp = np.zeros((self.num_xpts*self.num_ypts,3), np.float32)
        objp[:,:2] = np.mgrid[0:self.num_xpts,0:self.num_ypts].T.reshape(-1,2)
        
        # read images and find the chess board corner
        for img in images:
            # read image
            im = cv2.imread(img)
            # convert to gray scale
            gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            # Find the chessboard corners
            ret, corners = cv2.findChessboardCorners(gray, (self.num_xpts,self.num_ypts), None)
            
            # If found, add object points, image points
            if ret is True:
                objpoints.append(objp)
                imgpoints.append(corners)
                
                # draw and display the corners
                img = cv2.drawChessboardCorners(im, (self.num_xpts, self.num_ypts), corners, ret)
                cv2.imshow("image",img)
                cv2.waitKey(0)
            else:
                print("Warning: Could not find correct number of corners for image {}".format(img))
                
        img_size = (img.shape[1], img.shape[0])        
        # Get camera matrix and distortion coeff
        ret, self.mtx, self.dist, self.rvecs, self.tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
        cv2.destroyAllWindows()
        
    def undistort_image(self, img):
        dst = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
        return dst
    
    def set_debug_mode(self, mode):
        self.debug_mode = mode
    
    def threshold_image(self, img):
        # note the image should be un-distorted
        # first convert the image to HLS color space and choose S channel as it is invariant under different lighting conditions
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        s_channel = hls[:,:,2]
        if self.debug_mode:
            plt.figure(figsize=(8,8))
            plt.imshow(s_channel)
            plt.title('S channel')
            plt.show()
        
        # convert the RGB image to gray scale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # equalize the grayscale histogram
        gray = cv2.equalizeHist(gray)
        
        # compute Sobel X
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        abs_sobelx = np.absolute(sobelx)
        sobelx_int = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
        
        # create a binary image by thresholding the sobelx
        thresh_min = 30
        thresh_val = 255
        # the below function will replace any pixel value greater than thresh_min with thresh_val as pixel value
        ret, sobelx_binary_int = cv2.threshold(sobelx_int, thresh_min, thresh_val, cv2.THRESH_BINARY)
        sobelx_binary = np.zeros_like(sobelx_binary_int)
        sobelx_binary[(sobelx_binary_int==thresh_val)] = 1
        
        if self.debug_mode:
            plt.figure(figsize=(8,8))
            plt.imshow(img)
            plt.title('Original image')
            plt.show()
            plt.figure(figsize=(8,8))
            plt.imshow(sobelx_binary_int, cmap='gray')
            plt.title('SobelXBinary')
            plt.show()
            
        # s-channel thresholding
        sch_min_thresh = 175
        sch_max_thresh = 255
        sch_binary_int = cv2.inRange(s_channel, sch_min_thresh, sch_max_thresh)
        sch_binary = np.zeros_like(sch_binary_int)
        sch_binary[(sch_binary_int>=sch_min_thresh) & (sch_binary_int<=sch_max_thresh)] = 1
        
        if self.debug_mode:
            plt.figure(figsize=(8,8))
            plt.imshow(sch_binary_int, cmap='gray')
            plt.title('Sch-Binary')
            plt.show()
            
        # stack the Sch binary image and sobelX binary image along the depth dimension for individual visualization
        if self.debug_mode:
            color_binary = np.dstack((np.zeros_like(sobelx_binary_int), sobelx_binary_int, sch_binary_int))
            plt.figure(figsize=(8,8))
            plt.imshow(color_binary, cmap='gray')
            plt.title('Sch-color_binary')
            plt.show()
            
        # combine both binary image
        combined_binary = np.zeros_like(sobelx_binary)
        combined_binary[(sobelx_binary==1) | (sch_binary==1)] = 1
        
        if self.debug_mode:
            plt.figure(figsize=(8,8))
            plt.imshow(combined_binary.astype('uint8'), cmap='gray')
            plt.title('Combined-binary')
            plt.show()
        
        return combined_binary
            
    def get_camera_calib(self):
        # function is called when we need to save the camera calibration data to the disk
        return self.mtx, self.dist
    
    def set_camera_calib(self, mtx, dist):
        self.mtx = mtx
        self.dist = dist
        
    def perspective_transform(self,img):
        # Note input should be un-distorted image
        img_size = (img.shape[1], img.shape[0])
        perspective_img = cv2.warpPerspective(img, self.M, img_size, flags=cv2.INTER_LINEAR)
        return perspective_img
            
    

In [ ]:
# create camera class object
camera = Camera(num_xpts=6, num_ypts=9)

In [ ]:
# calibrate camera
path = './camera_cal/calibration*.jpg'
camera.caliberate_camera(path)

In [ ]:
# check the undistortion on test images
images = glob.glob(path)

for img in images:
    # read image
    im = cv2.imread(img)
    cv2.imshow("before calibration",im)
    # un distort the image
    undist = camera.undistort_image(im)
    cv2.imshow("after calibration",undist)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

In [ ]:
path = './test_images/*.jpg'
images = glob.glob(path)
#test the image binary thresholding
camera.set_debug_mode(mode=True)
img = cv2.imread(images[0])
# un-distort the image
img_undistort = camera.undistort_image(img)
binary_img = camera.threshold_image(img_undistort)

In [ ]:
mtx, dist =camera.get_camera_calib()
# save the camera matrix, distortion coeff to the disk
pickle_file = 'camera_calib.pickle'
if not os.path.isfile(pickle_file):
    print('Saving data to pickle file...')
    try:
        with open('camera_calib.pickle', 'wb') as pfile:
            pickle.dump(
                {
                    'camera_mat': mtx,
                    'dist_coeff': dist,
            
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

print('Data cached in pickle file.')

In [ ]:
# load the picke file
# Reload the data
pickle_file = 'camera_calib.pickle'
with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  mtx = pickle_data['camera_mat']
  dist = pickle_data['dist_coeff']
  del pickle_data  # Free up memory


print('Data and modules loaded.')
# set the camera matrix and distortion coeff
camera.set_camera_calib(mtx,dist)

In [ ]:
from win32api import GetSystemMetrics

#the [x, y] for each right-click event will be stored here
right_clicks = list()

#this function will be called whenever the mouse is right-clicked
def mouse_callback(event, x, y, flags, params):

    #right-click event value is 2
    if event == 2:
        global right_clicks

        #store the coordinates of the right-click event
        right_clicks.append([x, y])

        #this just verifies that the mouse data is being collected
        #you probably want to remove this later
        print(right_clicks)


In [ ]:
path = './test_images/*.jpg'
images = glob.glob(path)
im = cv2.imread(images[0])
# undistort the image
img_undistort = camera.undistort_image(im)
cv2.imshow('image',img_undistort)
#set mouse callback function for window
cv2.setMouseCallback('image', mouse_callback)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
src_cordinates = np.float32([[608, 445], [669, 441], [1091, 714], [226, 711]])
dest_cordinates = np.float32([[200,0],
                         [1080,0],
                         [1080,719],
                         [200,719]])
M = cv2.getPerspectiveTransform(src_cordinates, dest_cordinates)
img_size = (im.shape[1], im.shape[0])
perspective_img = cv2.warpPerspective(img_undistort, M, img_size, flags=cv2.INTER_LINEAR)
plt.figure(figsize=(8,8))
plt.imshow(perspective_img)
plt.show()

In [ ]:
# get the perpspective transform on the combined binary image
path = './test_images/*.jpg'
images = glob.glob(path)
im = cv2.imread(images[0])
# undistort the image
img_undistort = camera.undistort_image(im)
# get the binary image
img_binary = camera.threshold_image(img_undistort)
# perform perspective transform
img_persp = camera.perspective_transform(img_binary)
plt.figure(figsize=(8,8))
plt.imshow(img_persp,cmap='gray')
plt.show()
#img_persp
#cv2.imshow("image",img_persp)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [3]:
class Lanes():
    def __init__(self, debug_mode=False):
        self.frame_no = 0
        # locations from last frame
        self.xleft_prevframe = None
        self.xright_prevframe = None
        # locations from current frame
        self.xleft_currframe = None
        self.xright_currframe = None
        # selected x pixel for left/right lane
        self.leftx = None
        self.rightx = None
        # set debug mode
        self.debug_mode = debug_mode
        # line fit coefficients for the current frame
        self.left_fit = []
        self.right_fit = []
        self.fit_avglen = 5 # frame over which fit coeff are averaged
        # margin for search round the line fit or the peak histogram value for both  x and y direction
        self.margin = 100
        # number of pixels to be detected to re-center the window (used in first frame line detection)
        self.minpix = 50
        # Choose the number of sliding windows
        self.nwindows = 9
        # the below variables are used for plotting the lines and search region on the image
        self.left_lane_inds = []
        self.right_lane_inds = []
        # radius of curvature
        self.left_curverad = 0
        self.right_curverad = 0
        self.dist_frm_center = None
        # flag to trigger full scan
        self.triggerfulscan = 0
        # frame count since either no left or no right line detected
        self.numframe_no_l_r = None
        # threshold in terms of frame before a full scan is launched
        self.numframe_thresh = 5
        self.prev_result = None
        
        
    def locate_lines(self, binary_warped):
        if self.frame_no == 0 or self.triggerfulscan == 1:
            if self.triggerfulscan==1:
                print("...Fullscan triggered....")
                print("Frameno:{}".format(self.frame_no))
                self.numframe_no_l_r = 0
                self.triggerfulscan = 0
                self.left_fit = []
                self.left_fit = []
            # input should be undistorted and warped image (bird's eye view)
            # Take a histogram of the bottom half of the image
            histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
            # Create an output image to draw on and  visualize the result
            # out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
            # Find the peak of the left and right halves of the histogram
            # These will be the starting point for the left and right lines
            midpoint = np.int(histogram.shape[0]/2)
            leftx_base = np.argmax(histogram[:midpoint])
            rightx_base = np.argmax(histogram[midpoint:]) + midpoint
            # Choose the number of sliding windows
            nwindows = self.nwindows
            # Set height of windows
            window_height = np.int(binary_warped.shape[0]/nwindows)
            # Identify the x and y positions of all nonzero pixels in the image
            nonzero = binary_warped.nonzero()
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            # Current positions to be updated for each window
            self.xleft_currframe = leftx_base
            self.xright_currframe = rightx_base
            # Set the width of the windows +/- margin
            margin = self.margin
            # Set minimum number of pixels found to recenter window
            minpix = self.minpix
            # Create empty lists to receive left and right lane pixel indices
            left_lane_inds = []
            right_lane_inds = []
            # Step through the windows one by one
            for window in range(nwindows):
                # Identify window boundaries in x and y (and right and left)
                win_y_low = binary_warped.shape[0] - (window+1)*window_height
                win_y_high = binary_warped.shape[0] - window*window_height
                win_xleft_low = self.xleft_currframe - margin
                win_xleft_high = self.xleft_currframe + margin
                win_xright_low = self.xright_currframe - margin
                win_xright_high = self.xright_currframe + margin
                # Draw the windows on the visualization image
                #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
                #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
                # Identify the nonzero pixels in x and y within the window
                good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
                good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
                # Append these indices to the lists
                left_lane_inds.append(good_left_inds)
                right_lane_inds.append(good_right_inds)
                # If you found > minpix pixels, recenter next window on their mean position
                if len(good_left_inds) > minpix:
                    self.xleft_currframe = np.int(np.mean(nonzerox[good_left_inds]))
                if len(good_right_inds) > minpix:        
                    self.xright_currframe = np.int(np.mean(nonzerox[good_right_inds]))

            # Concatenate the arrays of indices
            left_lane_inds = np.concatenate(left_lane_inds)
            right_lane_inds = np.concatenate(right_lane_inds)

            # Extract left and right line pixel positions
            leftx = nonzerox[left_lane_inds]
            lefty = nonzeroy[left_lane_inds] 
            rightx = nonzerox[right_lane_inds]
            righty = nonzeroy[right_lane_inds] 

            # Fit a second order polynomial to each
            self.left_fit.append(np.polyfit(lefty, leftx, 2))
            self.right_fit.append(np.polyfit(righty, rightx, 2))
            
                        
        else:
            # Assume you now have a new warped binary image 
            # from the next frame of video (also called "binary_warped")
            # It's now much easier to find line pixels!
            nonzero = binary_warped.nonzero()
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            # set the fit parameters based on previous frame
            left_fit  = self.left_fit
            right_fit = self.right_fit
            margin    = self.margin
            # take the average
            left_fit = np.mean(left_fit, axis=0)
            right_fit = np.mean(right_fit, axis=0)
            left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
            right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  
            # Again, extract left and right line pixel positions
            leftx = nonzerox[left_lane_inds]
            lefty = nonzeroy[left_lane_inds] 
            rightx = nonzerox[right_lane_inds]
            righty = nonzeroy[right_lane_inds]
            
            
            if  (len(leftx)==0) or (len(rightx)==0):
                self.triggerfulscan = 1
            elif (np.max(leftx)>=np.min(rightx)):
                self.triggerfulscan = 1
            else:
                # Fit a second order polynomial to each
                left_fit = np.polyfit(lefty, leftx, 2)
                right_fit = np.polyfit(righty, rightx, 2)
                
                if len(self.left_fit)<self.fit_avglen:
                    self.left_fit.append(left_fit)
                else:
                    self.left_fit[0:self.fit_avglen-1] = self.left_fit[1:self.fit_avglen]
                    self.left_fit.append(left_fit)

                if len(self.right_fit)<self.fit_avglen:
                    self.right_fit.append(right_fit)
                else:
                    self.right_fit[0:self.fit_avglen-1] = self.right_fit[1:self.fit_avglen]
                    self.right_fit.append(right_fit)
                    # update the chosen left and right lane indices (used for plotting) for the current frame
                    self.left_lane_inds = left_lane_inds
                    self.right_lane_inds = right_lane_inds
                    
                
            # Fit a second order polynomial to each
            #left_fit = np.polyfit(lefty, leftx, 2)
            #right_fit = np.polyfit(righty, rightx, 2)
            # set the line fit cordinates for the current frame
            
            #if len(self.left_fit)<5:
            #    self.left_fit.append(left_fit)
            #else:
            #    self.left_fit[0:4] = self.left_fit[1:5]
            #    self.left_fit.append(left_fit)
                
            #if len(self.right_fit)<5:
            #    self.right_fit.append(right_fit)
            #else:
            #    self.right_fit[0:4] = self.right_fit[1:5]
            #    self.right_fit.append(right_fit)
        
        # increment the frame no
        self.frame_no += 1
        # update the chosen left and right lane indices (used for plotting) for the current frame
        #self.left_lane_inds = left_lane_inds
        #self.right_lane_inds = right_lane_inds
                        
        return leftx, lefty, rightx, righty
    
    def check_lanes(self):
        '''
        This function trigger a fresh full scan from next frame onwards if either left or right lane is not getting detected 
        for more than programmed threshold number of frames
        '''
        pass
        '''
        if sum(self.left_lane_inds) <= 50 or sum(self.right_lane_inds) <= 50:
            self.numframe_no_l_r += 1
            if self.numframe_no_l_r >= self.numframe_thresh:
                self.triggerfulscan = 1
                print("Setting full scan variable to 1...")
            else:
                self.triggerfulscan = 0
        '''
                    
        
    def plot_lanes(self, img, binary_warped, leftx, lefty, rightx, righty, Minv):
        '''
        img: un-warped original color image
        binary_warped : warped binary thresholded image
        nonzeroy : y indices for all non-zero pixels in binary thresholded warped image
        nonzerox : x indices for all non-zero pixels in binary thresholded warped image
        '''
        
        if self.triggerfulscan == 1:
            return self.prev_result
        
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        left_fit = np.mean(self.left_fit, axis=0)
        right_fit = np.mean(self.right_fit, axis=0)
        #print(left_fit)    
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
                
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        #window_img = np.zeros_like(out_img)
        
        #left_lane_inds  = self.left_lane_inds
        #right_lane_inds = self.right_lane_inds
        margin          = self.margin
        
        # Color in left and right line pixels
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(out_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(out_img, np.int_([right_line_pts]), (0,255, 0))
                
        # un-warp into original image space
        unwarp_newimg = cv2.warpPerspective(out_img, Minv, (img.shape[1], img.shape[0]))
        result = cv2.addWeighted(img, 1, unwarp_newimg, 0.3, 0)
        
        # compute ROC on current frame
        self.computeROC(binary_warped.shape[0], binary_warped.shape[1])
        
        # Write the radius of curvature for each lane 
        font = cv2.FONT_HERSHEY_SIMPLEX
        left_roc = "Roc: {0:.2f}m".format(self.left_curverad) 
        cv2.putText(result, left_roc, (10,650), font, 1, (255,255,255), 2)
        right_roc = "Roc: {0:.2f}m".format(self.right_curverad) 
        cv2.putText(result, right_roc, (1020,650), font, 1, (255,255,255), 2)
        dist_text = "Dist from Center: {0:.2f} cms".format(self.dist_frm_center)
        cv2.putText(result, dist_text, (450,50), font, 1, (255,255,255), 2)
                
        self.prev_result = result
        
        return result
    
    def computeROC(self, height, width):
        '''
        This function is to be called from locate_lines
        lefty  : y pixels chosen for line fit for left line in current frame
        righty : y pixels chosen for line fit for right line in current frame
        leftx  : x pixels chosen for line fit for left line in current frame
        rightx : x pixels chosen for line fit for right line in current frame
        height : height of the image
        '''
        ploty = np.linspace(0, 719, num=height)# to cover same y-range as image
        left_fit = np.mean(self.left_fit, axis=0)
        right_fit = np.mean(self.right_fit, axis=0)
        # find predictions for left and right x values
        leftx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2] 
        rightx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        
        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
        
        y_eval = np.max(ploty)
        # Calculate the new radii of curvature
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        #print("max yeval for right:{}".format(y_eval))
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        # Now our radius of curvature is in meters
        #print(left_curverad, 'm', right_curverad, 'm')
        self.left_curverad = left_curverad
        self.right_curverad = right_curverad
        # calculate distance from center in real world co-ordinates
        img_center = int(width/2)*xm_per_pix
        leftcord   = left_fit_cr[0]*(y_eval*ym_per_pix)**2 + left_fit_cr[1]*y_eval*ym_per_pix + left_fit_cr[2]
        rightcord  = right_fit_cr[0]*(y_eval*ym_per_pix)**2 + right_fit_cr[1]*y_eval*ym_per_pix + right_fit_cr[2]
        self.dist_frm_center = img_center - (rightcord + leftcord)/2
               
        
        
        

In [ ]:
# test the lanes detection
path = './test_images/*.jpg'
images = glob.glob(path)
im = cv2.imread(images[0])
# undistort the image
img_undistort = camera.undistort_image(im)
# get the binary image
img_binary = camera.threshold_image(img_undistort)
# perform perspective transform
img_persp = camera.perspective_transform(img_binary)

# instantiate object of class Lanes
lanes = Lanes()
lanes.locate_lines(img_persp)

In [4]:
## test scripts
test_images = True
test_video1 = False
test_video2 = False
test_video3 = False
dump_frames = True
def process_image(img):
    # undistort the image
    img_undistort = camera.undistort_image(img)
    # get the binary image
    img_binary = camera.threshold_image(img_undistort)
    # perform perspective transform
    img_persp = camera.perspective_transform(img_binary)
    # locate lanes
    leftx, lefty, rightx, righty = lanes.locate_lines(img_persp)
    # plot the lanes
    Minv = camera.Minv
    img_with_lanes = lanes.plot_lanes(img,img_persp,leftx,lefty,rightx,righty,Minv)
    # check lanes
    #lanes.check_lanes()
    
    
    if dump_frames:
        fname1 = './video_frames/'+ "frameno_"+ str(lanes.frame_no)+ 'undist'+ '.jpg'
        fname2 = './video_frames/'+ "frameno_"+ str(lanes.frame_no)+ 'binary'+ '.jpg'
        fname3 = './video_frames/'+ "frameno_"+ str(lanes.frame_no)+ 'persp'+ '.jpg'
        fname4 = './video_frames/'+ "frameno_"+ str(lanes.frame_no)+ 'withlanes'+ '.jpg'
        cv2.imwrite(fname1, img_undistort)
        mpimg.imsave(fname2, img_binary)
        mpimg.imsave(fname3, img_persp)
        cv2.imwrite(fname4, img_with_lanes)
        
    return img_with_lanes

In [ ]:
if test_images:
    save_images = True # all images will be saved
    path = './test_images/*.jpg'
    writepath = './output_images/'
    images = glob.glob(path)
    #images = images[0] # for debug
    
    # create camera class object
    camera = Camera(num_xpts=6, num_ypts=9)
    # create lanes class object
    lanes  = Lanes()
    # Reload the camera matrix and distortion coeff
    pickle_file = 'camera_calib.pickle'
    with open(pickle_file, 'rb') as f:
      pickle_data = pickle.load(f)
      mtx = pickle_data['camera_mat']
      dist = pickle_data['dist_coeff']
      del pickle_data  # Free up memory

    print('Data and modules loaded.')
    # set the camera matrix and distortion coeff
    camera.set_camera_calib(mtx,dist)
    
    # Setup the plot grid for test images
    plt.figure(figsize = (20,20))
    gs1 = gridspec.GridSpec(len(images),2)
    gs1.update(wspace=0.025, hspace=0.025)
    
    i=0
    
    for img in images:
        test_image = cv2.imread(img)
        img_withdetection = process_image(test_image)
        if save_images is True:
            # save the image
            y=img.split('\\')
            fname = writepath + y[1] 
            cv2.imwrite(fname,img_withdetection)
            
        b,g,r = cv2.split(img_withdetection)
        img_withdetection = cv2.merge((r,g,b))
        ax1 = plt.subplot(gs1[i])
        plt.axis('off')
        b,g,r = cv2.split(test_image)
        test_image = cv2.merge((r,g,b))
        ax1.imshow(test_image)
        ax2 = plt.subplot(gs1[i+1])
        plt.axis('off')
        ax2.imshow(img_withdetection)
        i+=2
 

In [ ]:
test_images = False
test_video1 = True
test_video2 = False
test_video3 = False
dump_frames = False
if test_video1:
    print("Running on test video1...")
    writepath = './output_images/'
    
    # create camera class object
    camera = Camera(num_xpts=6, num_ypts=9)
    # create lanes class object
    lanes  = Lanes()
    # Reload the camera matrix and distortion coeff
    pickle_file = 'camera_calib.pickle'
    with open(pickle_file, 'rb') as f:
      pickle_data = pickle.load(f)
      mtx = pickle_data['camera_mat']
      dist = pickle_data['dist_coeff']
      del pickle_data  # Free up memory

    print('Data and modules loaded.')
    # set the camera matrix and distortion coeff
    camera.set_camera_calib(mtx,dist)
    
    #####################################
    # Run our pipeline on the test video 
    #####################################
    clip = VideoFileClip("./project_video.mp4")
    output_video = "./project_video_processed_111.mp4"
    output_clip = clip.fl_image(process_image)
    output_clip.write_videofile(output_video, audio=False)

In [5]:
test_images = False
test_video1 = False
test_video2 = True
test_video3 = False
dump_frames = False
if test_video2:
    print("Running on test video2...")
    writepath = './output_images/'
    
    # create camera class object
    camera = Camera(num_xpts=6, num_ypts=9)
    # create lanes class object
    lanes  = Lanes()
    # Reload the camera matrix and distortion coeff
    pickle_file = 'camera_calib.pickle'
    with open(pickle_file, 'rb') as f:
      pickle_data = pickle.load(f)
      mtx = pickle_data['camera_mat']
      dist = pickle_data['dist_coeff']
      del pickle_data  # Free up memory

    print('Data and modules loaded.')
    # set the camera matrix and distortion coeff
    camera.set_camera_calib(mtx,dist)
    
    #####################################
    # Run our pipeline on the test video 
    #####################################
    clip = VideoFileClip("./challenge_video.mp4")
    output_video = "./project_video_processed_222.mp4"
    output_clip = clip.fl_image(process_image)
    output_clip.write_videofile(output_video, audio=False)

Running on test video2...
Data and modules loaded.
[MoviePy] >>>> Building video ./project_video_processed_222.mp4
[MoviePy] Writing video ./project_video_processed_222.mp4


 14%|█████▋                                   | 68/485 [00:09<01:05,  6.35it/s]

...Fullscan triggered....
Frameno:69


 17%|██████▊                                  | 81/485 [00:10<00:54,  7.39it/s]

...Fullscan triggered....
Frameno:82


...

...Fullscan triggered....
Frameno:276


 90%|███████████████████████████████████▉    | 435/485 [01:06<00:06,  7.72it/s]

...Fullscan triggered....
Frameno:436


100%|████████████████████████████████████████| 485/485 [01:13<00:00,  7.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./project_video_processed_222.mp4 



In [6]:
test_images = False
test_video1 = False
test_video2 = False
test_video3 = True
dump_frames = False
if test_video3:
    print("Running on test video3...")
    writepath = './output_images/'
    
    # create camera class object
    camera = Camera(num_xpts=6, num_ypts=9)
    # create lanes class object
    lanes  = Lanes()
    # Reload the camera matrix and distortion coeff
    pickle_file = 'camera_calib.pickle'
    with open(pickle_file, 'rb') as f:
      pickle_data = pickle.load(f)
      mtx = pickle_data['camera_mat']
      dist = pickle_data['dist_coeff']
      del pickle_data  # Free up memory

    print('Data and modules loaded.')
    # set the camera matrix and distortion coeff
    camera.set_camera_calib(mtx,dist)
    
    #####################################
    # Run our pipeline on the test video 
    #####################################
    clip = VideoFileClip("./harder_challenge_video.mp4")
    output_video = "./project_video_processed_333.mp4"
    output_clip = clip.fl_image(process_image)
    output_clip.write_videofile(output_video, audio=False)

Running on test video3...
Data and modules loaded.
[MoviePy] >>>> Building video ./project_video_processed_333.mp4
[MoviePy] Writing video ./project_video_processed_333.mp4


 42%|████████████████▍                      | 505/1200 [01:24<01:34,  7.39it/s]

...Fullscan triggered....
Frameno:506


 42%|████████████████▌                      | 508/1200 [01:24<01:37,  7.07it/s]

...Fullscan triggered....
Frameno:509


...

...Fullscan triggered....
Frameno:956


 84%|███████████████████████████████▊      | 1004/1200 [02:40<00:25,  7.79it/s]

...Fullscan triggered....
Frameno:1005


100%|█████████████████████████████████████▉| 1199/1200 [03:09<00:00,  6.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./project_video_processed_333.mp4 

